# Assignment 2

Importing Data

In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve
data = np.load('DATA/ct_data.npz')
X_train = data['X_train']; X_val = data['X_val']; X_test = data['X_test']
y_train = data['y_train']; y_val = data['y_val']; y_test = data['y_test']

In [2]:
print(f'X_train shape: {X_train.shape}')
print(f'y_train shape: {y_train.shape}')

X_train shape: (40754, 384)
y_train shape: (40754,)


Question 1a

In [3]:
#defining a function to calculate standard error
def cal_se(data):
    std = np.std(data)
    se = std/ (len(data)**(1/2))
    return se

#verifying the mean on the training set is zero
print(f'The mean on y_train is {round(np.mean(y_train), 8)}.')
print(f'The mean in y_val is {round(np.mean(y_val),8)} +/- {round(cal_se(y_val), 8)}.')

print(f'The mean on the first 5785 entries in y_train is {round(np.mean(y_train[:5785]), 8)} +/- {round(cal_se(y_train[:5785]),8)}.')
#used 8 decimal places as that is the pattern of the data

The mean on y_train is -0.0.
The mean in y_val is -0.21600851 +/- 0.01290338.
The mean on the first 5785 entries in y_train is -0.44247688 +/- 0.01192627.


The standard errors on the mean values of the first 5785 entries of training and the validation dataset suggest that even the edge cases of the mean of the two sets does not meet the overal population mean values. The standard error bars are misleading here because they do not align to what the population mean is??? 

standard error --> makes it look like we are more certain than we are. 

Question 1b

In [4]:
isConstant = []
isDuplicates = np.array(np.zeros(X_train.shape[1]), dtype='bool') 

#identify all the constant columns
for i in range(X_train.shape[1]) :
    isConstant.append((X_train[:,i] == X_train[0][i]).all())

#identify all the columns which are duplicates to previous columns
for i in range(X_train.shape[1]) :
    for k in range(i+1, X_train.shape[1]) :
        if isDuplicates[k] == False:
            isDuplicates[k] = (X_train[:,i] == X_train[:,k]).all()

#create a list with all the columns to remove
columns_to_remove = np.unique(np.hstack((np.nonzero(isDuplicates)[0], np.nonzero(isConstant)[0])))

#remove these columns and name them modified datasets
X_train = np.delete(X_train, columns_to_remove ,1)
X_val = np.delete(X_val, columns_to_remove ,1)
X_test = np.delete(X_test, columns_to_remove ,1)

#check if the shape still aligns to expectations (number of rows stay the same, number of columns are same for all three sets)
print(X_train.shape, X_val.shape, X_test.shape)
# print list of removed columns
print(f'constant columns: {np.nonzero(isConstant)[0]}')
print(f'duplicate columns: {np.nonzero(isDuplicates)[0]}')

(40754, 373) (5785, 373) (6961, 373)
constant columns: [ 59  69 179 189 351]
duplicate columns: [ 69  78  79 179 188 189 199 287 351 359]


Question 2

We want to set up the following matrix equation to solve for the weights and bias of the linear regression model.
$$ \tilde{\Phi}=\left[\begin{matrix} X_{N\times D} & 1_{N\times 1}\\ \sqrt{\alpha}\mathbb{I}_{D\times D} & 0_{D\times 1} \end{matrix}\right],\quad \underline{\tilde{w}}=\left[\begin{matrix} w_{D\times 1}\\ b\end{matrix}\right],\quad \underline{\tilde{y}}=\left[\begin{matrix} y_{D\times 1}\\ 0\end{matrix}\right] $$
$$ E(\underline{w},b)=(\tilde{\Phi}\underline{\tilde{w}}-\underline{\tilde{y}})^T(\tilde{\Phi}\underline{\tilde{w}}-\underline{\tilde{y}}) $$

In [5]:
def fit_linreg(X, yy, alpha):
    yy = yy[:, np.newaxis]
    # add a column of ones to the X matrix
    Phi = np.hstack((X, np.ones((X.shape[0],1))))
    # add an identity matrix to the Phi matrix for regularization
    # leave the last column as zeros to ignore the bias term
    Phi_til = np.vstack((Phi, np.hstack((np.sqrt(alpha)*np.eye(X.shape[1]), np.zeros((X.shape[1],1))))))
    # compute the new y vector
    yy_til = np.vstack((yy, np.zeros((X.shape[1],1))))
    # compute the weights
    w = np.linalg.lstsq(Phi_til, yy_til, rcond=None)[0][:,0]
    return w[:-1], w[-1]

In [6]:
# compute the weighs and bias for the linreg model
w_linreg, b_linreg = fit_linreg(X_train, y_train, 30)

In [7]:
from ct_support_code import fit_linreg_gradopt
# compute the weighs and bias for the linreg_gradopt model
w_grad, b_grad = fit_linreg_gradopt(X_train, y_train, 30)

In [8]:
def compute_rmse(X, yy, ww, bb):
    residuals = X @ ww[:, np.newaxis] + bb - yy[:, np.newaxis]
    return np.sqrt(residuals.T @ residuals / len(yy))[0][0]

In [9]:
print(f'RMSE for training linear regression: {compute_rmse(X_train, y_train, w_linreg, b_linreg)}')
print(f'RMSE for training gradient descent: {compute_rmse(X_train, y_train, w_grad, b_grad)}')
print(f'RMSE for validation linear regression: {compute_rmse(X_val, y_val, w_linreg, b_linreg)}')
print(f'RMSE for validation gradient descent: {compute_rmse(X_val, y_val, w_grad, b_grad)}')

RMSE for training linear regression: 0.3567565397204054
RMSE for training gradient descent: 0.35675704441316
RMSE for validation linear regression: 0.4230521968394691
RMSE for validation gradient descent: 0.42305140573952943


Question 3

In [10]:
from ct_support_code import logreg_cost, minimize_list
#write a function to fit logistc regresion using gradient opt
def fit_logreg_gradopt(X, yy, alpha):
    D = X.shape[1]
    args = (X, yy, alpha)
    init = (np.zeros(D), np.array(0))
    ww, bb = minimize_list(logreg_cost, init, args)
    return ww, bb

In [11]:
K = 20 # number of thresholded classification problems to fit
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
params = []
for kk in range(K):
    labels = y_train > thresholds[kk]
    # ... fit logistic regression to these labels
    params.append(fit_logreg_gradopt(X_train, labels, 30))

In [12]:
pred_list_train = []
pred_list_val = []

#sigmoid function to be used to normalise X
def sigmoid_func(x): 
    return 1/(1+np.exp(-x))

#utilise the parameters from the model fitted earlier and apply sigmoid on the predictions
for i in range(K):
    pred_list_train.append(sigmoid_func(X_train @ params[i][0] + params[i][1]))
    pred_list_val.append(sigmoid_func(X_val @ params[i][0] + params[i][1]))
    
X_train_transform = np.vstack(pred_list_train).T
X_val_transform = np.vstack(pred_list_val).T

In [13]:
#fit linear regression on the predictions
w_train, b_train = fit_linreg(X_train_transform, y_train, 30)

In [14]:
print(f'RMSE for training linear regression: {compute_rmse(X_train_transform, y_train, w_train, b_train)}')
print(f'RMSE for training linear regression: {compute_rmse(X_val_transform, y_val, w_train, b_train)}')

RMSE for training linear regression: 0.15441150429813472
RMSE for training linear regression: 0.2542477297925759


# Question 4

In [22]:
#fitting a neural networks with pre-determined weights 
from ct_support_code import nn_cost
def fit_nn(init, X, yy, alpha):
    args = (X, yy, alpha)
    ww_bar, bb_bar, V_bar, bk_bar = minimize_list(nn_cost, init, args)
    return ww_bar, bb_bar, V_bar, bk_bar

In [23]:
input_weights = np.vstack([params[k][0] for k in range(K)])
input_biases = np.array([params[k][1] for k in range(K)])
init = w_train, b_train, input_weights, input_biases

new_params = fit_nn(init, X_train, y_train, 30)

KeyboardInterrupt: 

In [ ]:
# fitting a neural networks with random weights

rand_hidden_weights = np.random.uniform(-1, 1, K)
rand_hidden_bias = np.random.uniform(-1, 1)
rand_input_weights = np.random.uniform(-1, 1, (K, X_train.shape[1]))
rand_input_bias = np.random.uniform(-1, 1, K)
rand_init = rand_hidden_weights, rand_hidden_bias, rand_input_weights, rand_input_bias

rand_nn_params = fit_nn(rand_init, X_train, y_train, 30)

In [ ]:
# make predictions
nn_train_predictions = nn_cost(new_params, X_train)
nn_val_predictions = nn_cost(new_params, X_val)
def nn_rmse(predictions, yy):
    residuals = (predictions - yy)[:,np.newaxis]
    return np.sqrt(residuals.T @ residuals / len(yy))[0][0]
print(f'RMSE for train NN: {nn_rmse(nn_train_predictions, y_train)}')
print(f'RMSE for val NN: {nn_rmse(nn_val_predictions, y_val)}')

RMSE for train NN: 0.13962174633354246
RMSE for val NN: 0.2684062894655103


In [ ]:
rand_nn_train_predictions = nn_cost(rand_nn_params, X_train)
rand_nn_val_predictions = nn_cost(rand_nn_params, X_val)
print(f'RMSE for train NN: {nn_rmse(rand_nn_train_predictions, y_train)}')
print(f'RMSE for val NN: {nn_rmse(rand_nn_val_predictions, y_val)}')

RMSE for train NN: 0.13930023727216795
RMSE for val NN: 0.2719298622850644


# Question 5

In [ ]:
def train_nn_reg(alpha, X_train, y_train, X_val, y_val):
    hidden_weights = np.random.uniform(-1, 1, K)
    hidden_bias = np.random.uniform(-1, 1)
    input_weights = np.random.uniform(-1, 1, (K, X_train.shape[1]))
    input_bias = np.random.uniform(-1, 1, K)
    init = hidden_weights, hidden_bias, input_weights, input_bias

    nn_params = fit_nn(init, X_train, y_train, alpha)
    val_predictions = nn_cost(nn_params, X_val)
    return nn_rmse(val_predictions, y_val)

In [ ]:
alpha_list = np.array([0,25,50])
baseline = np.log(nn_rmse(rand_nn_val_predictions, y_val))

In [ ]:
alpha_nn_rmse = np.array([train_nn_reg(alpha, X_train, y_train, X_val, y_val) for alpha in alpha_list])
alpha_nn_y_vals = baseline - np.log(alpha_nn_rmse)

In [ ]:
from ct_support_code import gp_post_par
alpha_all = np.arange(0, 50, 0.02)
alpha_rest = np.setdiff1d(alpha_all, alpha_list)
gp_post_par_resutlts = gp_post_par(alpha_rest, alpha_list, alpha_nn_y_vals)

In [ ]:
import scipy.stats as stats
def acquisition_function(gp_post_par_resutlts, y_obs):
    return stats.norm.cdf((gp_post_par_resutlts[0] - np.max(y_obs))/np.diag(gp_post_par_resutlts[1]))

In [ ]:
alpha_list = np.array([0,25,50])
alpha_nn_rmse = alpha_nn_rmse[:3]
alpha_nn_y_vals = alpha_nn_y_vals[:3]

In [ ]:
new_alphas = []
prob_imp_list = []
for i in range(5):
    prob_imp = acquisition_function(gp_post_par_resutlts, alpha_nn_y_vals)
    prob_imp_list.append(prob_imp[np.argmax(prob_imp)])
    new_alpha = alpha_all[np.argmax(prob_imp)]
    new_alphas.append(new_alpha)
    print(new_alpha)
    if i != 4:
        alpha_list = np.append(alpha_list, new_alpha)
        alpha_nn_rmse = np.append(alpha_nn_rmse, train_nn_reg(new_alpha, X_train, y_train, X_val, y_val))
        alpha_nn_y_vals = baseline - np.log(alpha_nn_rmse)
        gp_post_par_resutlts = gp_post_par(alpha_rest, alpha_list, alpha_nn_y_vals)

20.68


16.66
12.64
9.14
36.42


In [ ]:
print(prob_imp_list)
print(new_alphas)

[0.3113905214879238, 0.07923897635238492, 0.003945606495911536, 3.3341753499797556e-14, 1.7024546985393874e-23]
[20.68, 16.66, 12.64, 9.14, 36.42]


In [ ]:
best_index = np.argmax(prob_imp_list)
best_prob_imp = prob_imp_list[best_index]
best_alpha = new_alphas[best_index]
print(best_alpha, best_prob_imp)

20.68 0.3113905214879238


In [ ]:
def get_nn_RMSE(alpha, X_train, y_train, X_val, y_val):
    hidden_weights = np.random.uniform(-1, 1, K)
    hidden_bias = np.random.uniform(-1, 1)
    input_weights = np.random.uniform(-1, 1, (K, X_train.shape[1]))
    input_bias = np.random.uniform(-1, 1, K)
    init = hidden_weights, hidden_bias, input_weights, input_bias

    nn_params = fit_nn(init, X_train, y_train, alpha)
    train_predictions = nn_cost(nn_params, X_train)
    val_predictions = nn_cost(nn_params, X_val)
    print(f'RMSE for train NN: {nn_rmse(train_predictions, y_train)}')
    print(f'RMSE for val NN: {nn_rmse(val_predictions, y_val)}')

get_nn_RMSE(best_alpha, X_train, y_train, X_val, y_val)

RMSE for train NN: 0.12106856533283024
RMSE for val NN: 0.26104825630741324


# Question 6

In [67]:
# tunning the best k 
K_list = np.arange(10,31)
k_tunning_results = []

for K in K_list:

    mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
    thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
    params = []
    for kk in range(K):
        labels = y_train > thresholds[kk]
        # ... fit logistic regression to these labels
        params.append(fit_logreg_gradopt(X_train, labels, 30))

    pred_list_train = []
    pred_list_val = []

    #sigmoid function to be used to normalise X
    def sigmoid_func(x): 
        return 1/(1+np.exp(-x))

    #utilise the parameters from the model fitted earlier and apply sigmoid on the predictions
    for i in range(K):
        pred_list_train.append(sigmoid_func(X_train @ params[i][0] + params[i][1]))
        pred_list_val.append(sigmoid_func(X_val @ params[i][0] + params[i][1]))
        
    X_train_transform = np.vstack(pred_list_train).T
    X_val_transform = np.vstack(pred_list_val).T    

    #fit linear regression on the predictions
    w_train, b_train = fit_linreg(X_train_transform, y_train, 30)
    train_rmse = compute_rmse(X_train_transform, y_train, w_train, b_train)
    val_rmse = compute_rmse(X_val_transform, y_val, w_train, b_train)

    print(f'RMSE for training linear regression using K = {K}: {train_rmse}')
    print(f'RMSE for validation linear regression using K = {K}: {val_rmse}')
    k_tunning_results.append((K, train_rmse, val_rmse))



RMSE for training linear regression using K = 10: 0.16535613879572222
RMSE for validation linear regression using K = 10: 0.25517932306696745
RMSE for training linear regression using K = 11: 0.16227092552526698
RMSE for validation linear regression using K = 11: 0.2562588770565986
RMSE for training linear regression using K = 12: 0.16124738352291107
RMSE for validation linear regression using K = 12: 0.2634884269861179
RMSE for training linear regression using K = 13: 0.1599645656189831
RMSE for validation linear regression using K = 13: 0.2532120922733342
RMSE for training linear regression using K = 14: 0.15734860991637672
RMSE for validation linear regression using K = 14: 0.24867525239682137
RMSE for training linear regression using K = 15: 0.15637190445431332
RMSE for validation linear regression using K = 15: 0.25795910081294854
RMSE for training linear regression using K = 16: 0.15510273762278062
RMSE for validation linear regression using K = 16: 0.2583045195255074
RMSE for tr

In [80]:
best_k = np.argmin(np.array(k_tunning_results)[:,2])
print(f'The best K value is {k_tunning_results[best_k][0]}.')
print(f'The training RMSE is {k_tunning_results[best_k][1]} and the validation RMSE is {k_tunning_results[best_k][2]}.')


The best K value is 14.
The training RMSE is 0.15734860991637672 and the validation RMSE is 0.24867525239682137.


In [84]:
K = 20
mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
params = []
for kk in range(K):
    labels = y_train > thresholds[kk]
    # ... fit logistic regression to these labels
    params.append(fit_logreg_gradopt(X_train, labels, 30))

pred_list_train = []
pred_list_test = []

#sigmoid function to be used to normalise X
def sigmoid_func(x): 
    return 1/(1+np.exp(-x))

#utilise the parameters from the model fitted earlier and apply sigmoid on the predictions
for i in range(K):
    pred_list_train.append(sigmoid_func(X_train @ params[i][0] + params[i][1]))
    pred_list_test.append(sigmoid_func(X_test @ params[i][0] + params[i][1]))
        
X_train_transform = np.vstack(pred_list_train).T
X_test_transform = np.vstack(pred_list_test).T    

#fit linear regression on the predictions
w_train, b_train = fit_linreg(X_train_transform, y_train, 30)
test_rmse = compute_rmse(X_test_transform, y_test, w_train, b_train)
print(f'RMSE for testing linear regression using K = {K}: {test_rmse}')

RMSE for testing linear regression using K = 20: 0.28433088097744263


In [36]:
X_train.shape[1]

373

In [38]:
# perform PCA
val_rmse_list = []
dimensions = np.arange(1, X_train.shape[1]+1)
cov = np.cov(X_train.T)
cov.shape
eig_vals, eig_vecs = np.linalg.eig(cov)
# sort eigenvalues and eigenvectors
idx = eig_vals.argsort()[::-1]
eig_vals = eig_vals[idx]
eig_vecs = eig_vecs[:,idx]

for dims in dimensions:
    # compute the projection matrix
    P = eig_vecs[:,:dims]
    # project the data
    X_train_pca = X_train @ P
    X_val_pca = X_val @ P
    X_test_pca = X_test @ P

    K = 20 # number of thresholded classification problems to fit
    mx = np.max(y_train); mn = np.min(y_train); hh = (mx-mn)/(K+1)
    thresholds = np.linspace(mn+hh, mx-hh, num=K, endpoint=True)
    params = []
    for kk in range(K):
        labels = y_train > thresholds[kk]
        # ... fit logistic regression to these labels
        params.append(fit_logreg_gradopt(X_train_pca, labels, 30))

    pred_list_train = []
    pred_list_val = []

    #utilise the parameters from the model fitted earlier and apply sigmoid on the predictions
    for i in range(K):
        pred_list_train.append(sigmoid_func(X_train_pca @ params[i][0] + params[i][1]))
        pred_list_val.append(sigmoid_func(X_val_pca @ params[i][0] + params[i][1]))
        
    X_train_transform = np.vstack(pred_list_train).T
    X_val_transform = np.vstack(pred_list_val).T

    #fit linear regression on the predictions
    w_train, b_train = fit_linreg(X_train_transform, y_train, 30)
    val_rmse = compute_rmse(X_val_transform, y_val, w_train, b_train)
    val_rmse_list.append(val_rmse)

    print(f'RMSE for training linear regression dim={dims}: {compute_rmse(X_train_transform, y_train, w_train, b_train)}')
    print(f'RMSE for validation linear regression dim={dims}: {val_rmse}')

RMSE for training linear regression dim=1: 0.900315966266321
RMSE for validation linear regression dim=1: 0.9180916747768344
RMSE for training linear regression dim=2: 0.7965913238850874
RMSE for validation linear regression dim=2: 0.8659959293224119
RMSE for training linear regression dim=3: 0.5986141017043963
RMSE for validation linear regression dim=3: 0.5849448562821985
RMSE for training linear regression dim=4: 0.6333127563290216
RMSE for validation linear regression dim=4: 0.6053722351594901
RMSE for training linear regression dim=5: 0.6200747443566733
RMSE for validation linear regression dim=5: 0.6137501508891946
RMSE for training linear regression dim=6: 0.5317055153139747
RMSE for validation linear regression dim=6: 0.4704039154643645
RMSE for training linear regression dim=7: 0.3913229703528288
RMSE for validation linear regression dim=7: 0.33335374367313975
RMSE for training linear regression dim=8: 0.36542870183244164
RMSE for validation linear regression dim=8: 0.35984884

In [39]:
len(val_rmse_list)

373

In [40]:
print(np.argmin(val_rmse_list), np.min(val_rmse_list))

201 0.2533959693309171
